## 🎯 The Pitch

### Problem Statement

**Traditional restaurant reservation systems are frustrating:**
- Rigid forms that don't understand context
- Multiple steps to complete a simple action
- No natural interaction
- Users must adapt to the system, not vice versa

### Solution

**A multi-agent AI system that:**
- Understands natural language requests
- Routes queries to specialized agents
- Executes real actions (create reservations, query menus)
- Remembers conversation context
- Provides a human-like interaction

### Value Proposition

| Without AI Agent | With AI Agent |
|------------------|---------------|
| Fill forms manually | Natural conversation |
| Navigate menus | Just ask |
| Multiple page loads | One chat interface |
| Cold experience | Warm, personalized |

**Example:**
```
User: "I want to book a table for 4 tomorrow at 8pm"
Agent: "Perfect! I'll help you with that. What's your name?"
User: "John Smith"
Agent: "Great John! What's your phone and email?"
...
Agent: "✅ Reservation confirmed! Your token is ABC123"
```

## 🏗️ Architecture

### System Diagram

```
┌─────────────────────────────────────────────────────────────────┐
│                          USER                                   │
│                     (Angular Frontend)                          │
└────────────────────────┬────────────────────────────────────────┘
                         │ HTTP
                         ▼
┌─────────────────────────────────────────────────────────────────┐
│                    FastAPI Server                                │
│                   POST /chat endpoint                            │
└────────────────────────┬────────────────────────────────────────┘
                         │
                         ▼
            ┌────────────────────────┐
            │  🎯 ORCHESTRATOR       │
            │  Analyzes query        │
            │  Routes to agents      │
            └────────┬───────────────┘
                     │
        ┌────────────┼────────────┐
        ▼            ▼            ▼
┌──────────────┐ ┌──────────────┐ ┌──────────────┐
│ 📅 RESERVAS  │ │ 🍽️ MENUS     │ │ ℹ️ INFO      │
│              │ │              │ │              │
│ - Create     │ │ - List       │ │ - Hours      │
│ - Modify     │ │ - Recommend  │ │ - Location   │
│ - Cancel     │ │              │ │ - Navigation │
│ - Query      │ │              │ │              │
└──────┬───────┘ └──────┬───────┘ └──────────────┘
       │                │                
       └────────┬───────┘                
                │ MCP Tools              
                ▼                        
       ┌────────────────┐               
       │  Node.js API   │◄──► MySQL DB  
       └────────────────┘               
```

### Components

| Component | Technology | Purpose |
|-----------|------------|--------|
| Frontend | Angular 17 | User interface with chat |
| Agent | Python + FastAPI | Multi-agent system |
| LLM | Gemini 2.5 Flash | Natural language understanding |
| Backend | Node.js + Express | REST API |
| Database | MySQL | Data persistence |

## ✨ Features Implemented (3+ Key Concepts)

### 1. 🎭 Multi-Agent System with Orchestration

The system uses **4 specialized agents** coordinated by an orchestrator:

- **Orchestrator**: Analyzes user intent and routes to appropriate agent(s)
- **Reservations Agent**: Handles all booking operations
- **Menus Agent**: Provides menu information and recommendations
- **Info Agent**: General information and navigation

**Parallel execution** is supported when a query needs multiple agents.

In [ ]:
# Agent Types Definition
from enum import Enum

class AgentType(Enum):
    """Types of available agents"""
    RESERVAS = "reservas"      # Booking operations
    MENUS = "menus"            # Menu information
    INFO = "info"              # General info
    ORCHESTRATOR = "orchestrator"  # Coordinator

class AgentStatus(Enum):
    """Agent execution states"""
    IDLE = "idle"
    RUNNING = "running"
    WAITING = "waiting"
    ERROR = "error"
    COMPLETED = "completed"

In [ ]:
# Multi-Agent Runner - Parallel Execution
import asyncio
from typing import Dict, List, Any, Optional

class MultiAgentRunner:
    """
    Manages multiple specialized agents.
    Supports parallel execution for multi-topic queries.
    """
    
    def __init__(self):
        self.agents: Dict[str, 'AgentRunner'] = {}
    
    def register_agent(self, agent: 'AgentRunner'):
        """Register a new agent in the system"""
        self.agents[agent.agent_id] = agent
    
    async def execute_parallel(self, tasks: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Execute multiple agents in parallel.
        Used when user asks about multiple topics.
        
        Example: "What menu do you recommend and what are the hours?"
        → Runs menus_agent AND info_agent simultaneously
        """
        async_tasks = [
            self.execute_agent(
                task['agent_id'],
                task['message'],
                task.get('context')
            )
            for task in tasks
        ]
        
        results = await asyncio.gather(*async_tasks, return_exceptions=True)
        return results

### 2. 🔧 Custom Tools (MCP Tools)

The agents can execute **real actions** through custom tools that interact with the backend API:

In [ ]:
# MCP Tools Definition
import httpx
from typing import Dict, Any, Optional

class RestauranteTools:
    """Custom tools for the restaurant agent"""
    
    def __init__(self):
        self.api_url = "http://localhost:3000/api"
    
    async def crear_reserva(
        self,
        nombre_cliente: str,
        telefono_cliente: str,
        email_cliente: str,
        fecha_reserva: str,
        num_personas: int,
        notas: Optional[str] = None
    ) -> Dict[str, Any]:
        """
        Creates a new reservation in the system.
        Returns reservation details and unique token.
        """
        data = {
            "nombre_cliente": nombre_cliente,
            "telefono_cliente": telefono_cliente,
            "email_cliente": email_cliente,
            "fecha_reserva": fecha_reserva,
            "num_personas": num_personas
        }
        if notas:
            data["notas"] = notas
        
        async with httpx.AsyncClient(timeout=10.0) as client:
            response = await client.post(f"{self.api_url}/reservas", json=data)
            if response.status_code == 201:
                result = response.json()
                return {
                    "success": True,
                    "reserva": result.get('reserva'),
                    "token": result.get('token'),
                    "mensaje": "Reservation created successfully!"
                }
            else:
                return {"success": False, "error": "Failed to create reservation"}

# Tool definitions for Gemini
TOOLS_DEFINITIONS = [
    {
        "name": "crear_reserva",
        "description": "Creates a new reservation. Requires all customer data.",
        "parameters": {
            "type": "object",
            "properties": {
                "nombre_cliente": {"type": "string", "description": "Customer full name"},
                "telefono_cliente": {"type": "string", "description": "Phone number"},
                "email_cliente": {"type": "string", "description": "Valid email"},
                "fecha_reserva": {"type": "string", "description": "Date in YYYY-MM-DDTHH:mm format"},
                "num_personas": {"type": "integer", "description": "Number of people (1-20)"}
            },
            "required": ["nombre_cliente", "telefono_cliente", "email_cliente", "fecha_reserva", "num_personas"]
        }
    },
    {
        "name": "consultar_reserva",
        "description": "Query reservation status using token",
        "parameters": {
            "type": "object",
            "properties": {
                "token": {"type": "string", "description": "Unique reservation token"}
            },
            "required": ["token"]
        }
    },
    {
        "name": "cancelar_reserva",
        "description": "Cancel an existing reservation",
        "parameters": {
            "type": "object",
            "properties": {
                "token": {"type": "string", "description": "Reservation token"}
            },
            "required": ["token"]
        }
    },
    {
        "name": "get_menu_mas_valorado",
        "description": "Gets the highest rated menu",
        "parameters": {}
    },
    {
        "name": "listar_menus_disponibles",
        "description": "Lists all available menus",
        "parameters": {}
    }
]

### 3. 💾 Sessions & Memory

Each agent maintains conversation history for context awareness:

In [ ]:
# Agent Runner with Memory
from datetime import datetime
import google.generativeai as genai

class AgentRunner:
    """
    Base class for running AI agents.
    Maintains conversation history for context.
    """
    
    def __init__(
        self,
        agent_id: str,
        agent_type: AgentType,
        model: genai.GenerativeModel,
        tools: Optional[List[Dict]] = None
    ):
        self.agent_id = agent_id
        self.agent_type = agent_type
        self.model = model
        self.tools = tools or []
        self.status = AgentStatus.IDLE
        
        # Session & Memory
        self.chat_history = []  # Conversation memory
        self.execution_count = 0
        self.created_at = datetime.now()
        self.last_execution = None
    
    async def execute(self, user_message: str, context: Optional[Dict] = None) -> Dict[str, Any]:
        """
        Execute agent with message.
        Uses chat history for context awareness.
        """
        self.status = AgentStatus.RUNNING
        self.execution_count += 1
        self.last_execution = datetime.now()
        
        # Create or continue chat with history
        if not hasattr(self, 'chat'):
            self.chat = self.model.start_chat(
                history=self.chat_history,
                enable_automatic_function_calling=True
            )
        
        # Send message
        response = self.chat.send_message(user_message)
        
        # Handle function calls if any
        response = await self._handle_function_calls(response)
        
        # Save to history (MEMORY)
        self.chat_history.append({"role": "user", "parts": [user_message]})
        self.chat_history.append({"role": "model", "parts": [response.text]})
        
        self.status = AgentStatus.COMPLETED
        
        return {
            "success": True,
            "agent_id": self.agent_id,
            "response": response.text,
            "execution_count": self.execution_count
        }
    
    def reset(self):
        """Reset agent memory"""
        self.chat_history = []
        self.status = AgentStatus.IDLE
        if hasattr(self, 'chat'):
            delattr(self, 'chat')

### 4. 🎯 Orchestrator Logic

The orchestrator analyzes user intent and routes to the appropriate agent(s):

In [ ]:
# Orchestrator System Prompt
ORCHESTRATOR_PROMPT = """
You are the Main Orchestrator for a restaurant multi-agent system.

AVAILABLE AGENTS:
1. reservas_agent: Handles reservations (create, modify, cancel, query)
2. menus_agent: Menu information, dishes, prices, recommendations
3. info_agent: General info (hours, location, policies), navigation

ROUTING RULES:
- "reservation", "book", "cancel", "token" → reservas_agent
- "menu", "dishes", "food", "recommend" → menus_agent  
- "hours", "location", "info" → info_agent
- Navigation requests → info_agent
- Ambiguous/greeting → info_agent
- Multiple topics → invoke multiple agents

RESPONSE FORMAT (JSON):
{
  "agents": ["agent_id1", "agent_id2"],
  "reasoning": "Brief explanation"
}
"""

# Example routing
example_queries = [
    ("I want to make a reservation", ["reservas_agent"]),
    ("What menu do you recommend?", ["menus_agent"]),
    ("What are your hours?", ["info_agent"]),
    ("What menu do you recommend and when do you open?", ["menus_agent", "info_agent"])  # Parallel!
]

for query, expected_agents in example_queries:
    print(f"Query: '{query}'")
    print(f"  → Routes to: {expected_agents}")
    print()

## 🔧 Technical Details

### Model Configuration

In [ ]:
# Gemini Model Configuration
import google.generativeai as genai

# For specialized agents
GENERATION_CONFIG = {
    "temperature": 0.7,      # Balance creativity/coherence
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 2048
}

# For orchestrator (more deterministic)
ORCHESTRATOR_CONFIG = {
    "temperature": 0.3,      # More consistent routing
    "max_output_tokens": 512
}

# Safety settings
SAFETY_SETTINGS = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"}
]

# Create agent model (example)
def create_agent_model(system_prompt: str, tools: list = None):
    return genai.GenerativeModel(
        model_name="gemini-2.5-flash",  # Using Gemini! (bonus points)
        generation_config=GENERATION_CONFIG,
        safety_settings=SAFETY_SETTINGS,
        system_instruction=system_prompt,
        tools=[{"function_declarations": tools}] if tools else []
    )

### FastAPI Server

In [ ]:
# Main FastAPI Application
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import List, Optional

app = FastAPI(
    title="Restaurant Multi-Agent System",
    description="AI-powered restaurant assistant with specialized agents",
    version="2.0.0"
)

# CORS for frontend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:4200"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"]
)

class Message(BaseModel):
    role: str
    content: str

class ChatRequest(BaseModel):
    messages: List[Message]
    session_id: Optional[str] = None

class ChatResponse(BaseModel):
    response: str
    session_id: Optional[str] = None
    navigation_action: Optional[str] = None

@app.post("/chat", response_model=ChatResponse)
async def chat(request: ChatRequest):
    """
    Main chat endpoint.
    Processes messages through the multi-agent system.
    """
    user_message = request.messages[-1].content
    
    # Process with multi-agent system
    result = await multi_agent_system.process_message(
        user_message=user_message,
        session_id=request.session_id
    )
    
    return ChatResponse(
        response=result.get("response", ""),
        session_id=result.get("session_id"),
        navigation_action=result.get("navigation_action")
    )

@app.get("/agents/status")
async def agents_status():
    """Get status of all agents in the system"""
    return multi_agent_system.get_system_status()

## 📊 System Status Example

The system provides observability through status endpoints:

In [ ]:
# Example system status response
example_status = {
    "total_agents": 4,
    "agents": {
        "orchestrator": {
            "agent_id": "orchestrator",
            "agent_type": "orchestrator",
            "status": "idle",
            "execution_count": 15,
            "history_length": 30
        },
        "reservas_agent": {
            "agent_id": "reservas_agent",
            "agent_type": "reservas",
            "status": "completed",
            "execution_count": 8,
            "history_length": 16
        },
        "menus_agent": {
            "agent_id": "menus_agent",
            "agent_type": "menus",
            "status": "idle",
            "execution_count": 5,
            "history_length": 10
        },
        "info_agent": {
            "agent_id": "info_agent",
            "agent_type": "info",
            "status": "idle",
            "execution_count": 12,
            "history_length": 24
        }
    },
    "timestamp": "2025-11-28T10:30:00"
}

import json
print(json.dumps(example_status, indent=2))

## 🎓 Reflection & Learnings

### What I Learned

| Day | Topic | Application in Project |
|-----|-------|------------------------|
| 1-2 | LLM Fundamentals | Understanding how Gemini processes natural language |
| 3 | Function Calling | Implemented MCP Tools for real actions |
| 4 | Multi-Agent Architecture | Created orchestrator + 3 specialized agents |
| 5 | Evaluation | Testing agent responses and routing |

### Challenges Faced

1. **Date Interpretation**: User says "tomorrow" but agent needs `YYYY-MM-DDTHH:mm`. Solved by injecting current date into the agent's system prompt.

2. **Agent Coordination**: Sometimes multiple agents wanted to respond. The orchestrator now uses JSON output format for clear routing decisions.

3. **Data Validation**: Agent must collect all required fields before creating a reservation. Implemented specific prompts that guide the conversation.

### Future Improvements

- 🔴 Session persistence with Redis
- 🔴 Automated evaluation metrics
- 🔴 Additional agents (payments, special events)
- 🔴 Voice interface

## 📚 Repository Links

- **Agent (Python)**: https://github.com/AxoneDesktop/Proyecto-4-Un-asistente-para-la-empresa-Agente
- **Backend (Node.js)**: https://github.com/AxoneDesktop/Proyecto-4-Un-asistente-para-la-empresa-backend
- **Frontend (Angular)**: https://github.com/AxoneDesktop/Proyecto-4-Un-asistente-para-la-empresa-frontend

---

## ✅ Summary of Key Concepts Used

| Concept | Implemented | Evidence |
|---------|-------------|----------|
| Multi-agent system | ✅ | 4 agents with orchestrator |
| Parallel agents | ✅ | `execute_parallel()` method |
| Sequential agents | ✅ | Orchestrator → Specialist |
| Custom Tools | ✅ | MCP Tools for reservations/menus |
| Sessions & Memory | ✅ | `chat_history` per agent |
| Gemini (bonus) | ✅ | `gemini-2.5-flash` model |

**Total: 6 concepts (minimum required: 3)** ✅